In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cheer_utils

dropcols=['SID','DIST2LAND','LANDFALL','USA_LAT','USA_LON','USA_STATUS','USA_AGENCY','IFLAG',
          'TRACK_TYPE','WMO_AGENCY','WMO_WIND','WMO_PRES','USA_SEAHGT','USA_SEARAD_SW',
          'USA_SEARAD_NW','USA_SEARAD_NE','USA_SEARAD_SE'];

renamecols={'SEASON': 'Year', 'LAT':'Latitude', 'LON':'Longitude','USA_PRES': 'Min_pres'}

fl='https://tdsres.apps.renci.org/thredds/fileServer/datalayers/ibtracs/ibtracs.NA.list.v04r00.csv'
df=pd.read_csv(fl,skiprows = [1]).drop(dropcols,axis=1).replace(' ', np.nan)
df=df[df.columns.drop(list(df.filter(regex='TOKYO|BOM|REUNION|WELLINGTON|CMA|NEUMANN|TD9636|DS824|NEWDELHI|HKO|TD9635|MLC|NADI')))]
df.USA_PRES=pd.to_numeric(df.USA_PRES)
df.USA_WIND=pd.to_numeric(df.USA_WIND)
df.USA_RMW=pd.to_numeric(df.USA_RMW)
df.rename(columns=renamecols,inplace=True)
df=df[df['BASIN']!='NI']
df=df[df['BASIN']!='EP']
df['BASIN']='NA'

In [ ]:
# add an "absolute storm number for keeping track of complete storms
#df['SEASON']*100+df['NUMBER']

df2=df.copy()
df2['abssn'] = -1
#print(df['abssn'])
#df.set_index('abssn',inplace=True)
#df[['abssn','USA_ATCF_ID']][90:110]
#ids
unique_ids=df2['USA_ATCF_ID'].unique()
for i,j in enumerate(unique_ids):
    idx=df2['USA_ATCF_ID']==j
    df2['abssn'][idx]=i
df=df2
df.set_index('abssn',inplace=True)

In [ ]:
# plot all tracks
fig = plt.figure(figsize=(16, 10))

ax = plt.axes(projection=ccrs.PlateCarree())
ax.stock_img()
ax.coastlines()
ax.set_xlim([-100, 0])
ax.set_ylim([0, 60])

#df.plot.scatter(x='LON',
#                y='LAT',
#                c='USA_WIND',
#                ax=ax)
for i,idx in enumerate(np.unique(df.index)): 
        x=df.loc[df.index==idx].Longitude
        y=df.loc[df.index==idx].Latitude
        c=df.loc[df.index==idx].Min_pres
        #c=df.loc[df.index==idx].HollandB
        
        ax.plot(x, y, linewidth=1, color='k')
        #axx.plot(x.iloc[0], y.iloc[0], marker='*', color='g')
        #axx.plot(x.iloc[-1], y.iloc[-1], marker='*', color='r')
        #cm=ax.scatter(x=x, y=y, c=c, cmap=cheer_utils.cmap, norm=cheer_utils.norm, s=36)

In [ ]:
# define dicts for filtering circles for case study areas:
a=np.arange(0,370,10)*np.pi/180
rad=3
lo=-76.5
la=34.75
# test track screening circle
# rad= .3
# lo=-77.75
# la=35
nc_circ={'lo': lo,
         'la': la, 
         'rad': rad,
         'cirx': rad*np.cos(a)+lo,
         'ciry': rad*np.sin(a)+la}

rad=3
lo=-94
la=29.5
tx_circ={'lo': lo,
         'la':  la,
         'rad': rad,
         'cirx': rad*np.cos(a)+lo,
         'ciry': rad*np.sin(a)+la}
         
# add distances to targets
df['dist2nc']=np.sqrt(np.square(df['Longitude']-nc_circ['lo']) + np.square(df['Latitude']-nc_circ['la']))
df['dist2tx']=np.sqrt(np.square(df['Longitude']-tx_circ['lo']) + np.square(df['Latitude']-tx_circ['la']))
df

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=2,
                           subplot_kw={'projection': ccrs.PlateCarree()},
                           figsize=(16,7))
ax=ax.flatten()
    
ax[0].stock_img()
ax[0].coastlines()
ax[1].stock_img()
ax[1].coastlines()
    
ax[0].plot(nc_circ['cirx'],nc_circ['ciry'],transform=ccrs.PlateCarree())
ax[1].plot(tx_circ['cirx'],tx_circ['ciry'],transform=ccrs.PlateCarree())

ax[0].plot(nc_circ['lo'],nc_circ['la'],marker='*',transform=ccrs.PlateCarree())
ax[1].plot(tx_circ['lo'],tx_circ['la'],marker='*',transform=ccrs.PlateCarree())

for i in range(0,2):
    ax[i].axis('equal')
    ax[i].set_ylim([10, 50])
    ax[i].set_xlim([-100, -50])
    ax[i].grid(True)

In [ ]:
# Find the storm index numbers (anssn) that pass thru the screening circles
# This gets the parts of tracks that enter the circles.  The complete track will be extracted below
df_nc=df.loc[(df['dist2nc'] <= nc_circ['rad']) & (df['Year'] >= 1980) & (df['Year'] <= 2020)]

#df_tx=df.loc[(df['dist2tx'] <= tx_circ['rad'])]# & (df['Min_pres'] <= 980)]
# df_nc=df.loc[(df['dist2nc'] <=nc_circ['rad']) & (df['Category'] > 2)]
# df_tx=df.loc[(df['dist2tx'] <=tx_circ['rad']) & (df['Category'] > 2)]
#idx_nc=np.unique(df_nc.index).astype(int)

idx_nc = df_nc.index.unique()
#idx_tx=np.unique(df_tx.index).astype(int)

print(f'Initial number of NC events = {len(idx_nc)}')
print(idx_nc)

#print(f'Initial number of TX events = {len(idx_tx)}')
df_nc

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=2,
                           subplot_kw={'projection': ccrs.PlateCarree()},
                           figsize=(16,7))
ax=ax.flatten()
    
ax[0].stock_img()
ax[0].coastlines()
ax[1].stock_img()
ax[1].coastlines()
    
ax[0].plot(nc_circ['cirx'],nc_circ['ciry'],transform=ccrs.PlateCarree())
for i,idx in enumerate(idx_nc): 
        x=df.loc[df.index==idx].Longitude
        y=df.loc[df.index==idx].Latitude
        c=df.loc[df.index==idx].Min_pres
        #c=df.loc[df.index==idx].HollandB
        
        ax[0].plot(x, y, linewidth=1, color='k')

for i in range(0,2):
    ax[i].axis('equal')
    ax[i].set_ylim([10, 50])
    ax[i].set_xlim([-100, -50])
    ax[i].grid(True)